# Week 3 Lecture 1

## Introduction to Data Manipulation with Pandas

This notebook is structured around 8 key operations you'll need to master:

- Reading the data
- Selecting the data
- Filtering the data
- Manipulating the data
- Sorting the data
- Grouping the data
- Rearranging the data
- Ranking the data

In [1]:
#import libraries
import numpy as np
import pandas as pd

In [2]:
# Create a sample DataFrame (simulating sales data)
np.random.seed(42)  # For reproducible results
data = {
    'Region': np.random.choice(['North', 'South', 'East', 'West'], 100),
    'Product': np.random.choice(['Widget A', 'Widget B', 'Widget C'], 100),
    'Sales': np.random.uniform(100, 1000, 100).round(2),
    'Quantity': np.random.randint(1, 20, 100),
    'Date': pd.date_range('2023-01-01', periods=100, freq='D'),
    'Discount': np.random.choice([0, 0.05, 0.10, 0.15], 100)
}
df = pd.DataFrame(data)
df['Revenue'] = df['Sales'] * df['Quantity'] * (1 - df['Discount'])  # Derived column

In [3]:
print("Sample DataFrame shape:", df.shape)
df.head()

Sample DataFrame shape: (100, 7)


,Region,Product,Sales,Quantity,Date,Discount,Revenue
0,East,Widget C,510.88,14,2023-01-01,0.05,6794.704
1,West,Widget B,296.60,16,2023-01-02,0.00,4745.600
2,North,Widget B,474.86,8,2023-01-03,0.05,3608.936
3,East,Widget B,894.95,7,2023-01-04,0.10,5638.185
4,East,Widget B,391.91,3,2023-01-05,0.10,1058.157


Note: This DataFrame has 100 rows. Feel free to run `df.info()` or `df.describe()` to explore it further.

### 1. Reading the Data

The first step in any data science project is loading data from external sources (e.g., CSV, Excel, JSON) into a pandas DataFrame. Pandas provides flexible `read_*` functions to handle various formats. If your data is already in memory, you can create a DataFrame directly from dictionaries, lists, or even NumPy arrays.

Key Tips:

- Always specify data types if possible (e.g., `dtype={'Sales': float}`) to avoid surprises.
- Handle missing values with `na_values` or `keep_default_na=False`.
- For large files, use `low_memory=False` to read in chunks.

#### Example 1: Creating a DataFrame from a Dictionary (In-Memory Data)
This is great for quick prototyping without files.

In [4]:
# From dictionary (like our sample above)
data_dict = {
    'Region': ['North', 'South'],
    'Sales': [100.0, 200.0]
}
df_from_dict = pd.DataFrame(data_dict)
print(df_from_dict)

  Region  Sales
0  North  100.0
1  South  200.0


#### Example 2: Reading from CSV (Assuming a File)
Save our sample to CSV first, then read it back. In practice, replace with your file path.

In [6]:
# Save sample to CSV (for demo)
df.to_csv('sample_sales.csv', index=False)

In [7]:
# Read from CSV
df_from_csv = pd.read_csv('sample_sales.csv', parse_dates=['Date'])  # Auto-parse dates
print("Shape from CSV:", df_from_csv.shape)
df_from_csv.head(2)

Shape from CSV: (100, 7)


,Region,Product,Sales,Quantity,Date,Discount,Revenue
0,East,Widget C,510.88,14,2023-01-01,0.05,6794.704
1,West,Widget B,296.60,16,2023-01-02,0.00,4745.600


Alternative Ways:

- Skip rows: `pd.read_csv('file.csv', skiprows=2)`
- Specify columns: `pd.read_csv('file.csv', usecols=['Region', 'Sales'])`
- Chunks for large files: `for chunk in pd.read_csv('file.csv', chunksize=1000): ...`

#### Example 3: Reading from Excel
Pandas can read Excel sheets directly (requires `openpyxl` or `xlrd`).

In [9]:
# Save to Excel (for demo)
df.to_excel('sample_sales.xlsx', index=False, sheet_name='Sales')

In [10]:
# Read from Excel
df_from_excel = pd.read_excel('sample_sales.xlsx', sheet_name='Sales', parse_dates=['Date'])
print("Shape from Excel:", df_from_excel.shape)
df_from_excel.head(2)

Shape from Excel: (100, 7)


,Region,Product,Sales,Quantity,Date,Discount,Revenue
0,East,Widget C,510.88,14,2023-01-01,0.05,6794.704
1,West,Widget B,296.60,16,2023-01-02,0.00,4745.600


Alternative Ways:

- Multiple sheets: `pd.read_excel('file.xlsx', sheet_name=None)` (returns dict of DataFrames)
- Specific range: `pd.read_excel('file.xlsx', usecols='A:C', nrows=5)`

#### Example 4: From JSON

__How JSON is Used in Data Science__
JSON is ubiquitous in data science because data rarely comes in perfectly clean, rectangular CSV files. Data scientists work with JSON data from many sources.

- __Web Scraping__: Data is often embedded in websites as JSON, especially on dynamic, modern sites.

- __API Interactions__: This is the primary source. As we'll see below, most APIs return data in JSON format. A data scientist writes scripts to call these APIs and collect JSON responses.

- __NoSQL Databases__: Databases like MongoDB store data directly in a JSON-like format (BSON).

An API (Application Programming Interface) is a set of rules and protocols that allows different software applications to communicate with each other.

In [11]:
# Sample JSON data
json_data = '{"Region": ["North"], "Sales": [150.0]}'
df_from_json = pd.read_json(json_data)
print(df_from_json)

  Region  Sales
0  North    150


C:\Users\hp\AppData\Local\Temp\ipykernel_7988\310397623.py:3: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_from_json = pd.read_json(json_data)


Alternative: pd.json_normalize(json_data) for nested JSON.

__Practice Exercise__: Create your own small dataset as a list of lists and convert it to a DataFrame using pd.DataFrame(data_list).

### 2. Selecting the Data
Explanation: Selecting data means extracting specific rows, columns, or subsets from your DataFrame. Use labels (column names) or positions (indices). This is foundational—avoid modifying the original data unless intended.
Key Tips:

Use `copy()` to create a safe copy: `subset = df.copy()`.
Chaining is efficient: `df['col1'].head()`.
For single values: `.at[]` (label-based, fast) vs `.iat[]` (position-based).

#### Example 1: Selecting Columns by Name
Basic way to pick one or more columns.

In [12]:
# Single column as Series
region_series = df['Region']
print("Single column:\n", region_series.head())

# Multiple columns as DataFrame
subset_cols = df[['Region', 'Sales', 'Revenue']]
print("\nMultiple columns shape:", subset_cols.shape)
subset_cols.head(2)

Single column:
 0     East
1     West
2    North
3     East
4     East
Name: Region, dtype: object

Multiple columns shape: (100, 3)


,Region,Sales,Revenue
0,East,510.88,6794.704
1,West,296.60,4745.600


Alternative Ways:

List of names: `df[['col1', 'col2']]`
Slice with `iloc: df.iloc[:, [0, 2]]`(first and third columns by position)

#### Example 2: Selecting Rows by Position (iloc)
Zero-based indexing for rows/columns.

In [13]:
# First 5 rows, all columns
first_five = df.iloc[0:5]
print(first_five)

# Specific rows and columns: rows 1-3, columns 0 and 2
specific = df.iloc[1:4, [0, 2]]
print("\nSpecific subset:\n", specific)

  Region   Product   Sales  Quantity       Date  Discount   Revenue
0   East  Widget C  510.88        14 2023-01-01      0.05  6794.704
1   West  Widget B  296.60        16 2023-01-02      0.00  4745.600
2  North  Widget B  474.86         8 2023-01-03      0.05  3608.936
3   East  Widget B  894.95         7 2023-01-04      0.10  5638.185
4   East  Widget B  391.91         3 2023-01-05      0.10  1058.157

Specific subset:
   Region   Sales
1   West  296.60
2  North  474.86
3   East  894.95


#### Example 3: Selecting Rows by Label (loc)
Uses index labels (default is integers).

In [14]:
# Rows where index is 0-4, columns 'Region' and 'Sales'
loc_subset = df.loc[0:4, ['Region', 'Sales']]
print(loc_subset)

# Single value: Sales for row 0
single_val = df.loc[0, 'Sales']
print("\nSingle value:", single_val)

  Region   Sales
0   East  510.88
1   West  296.60
2  North  474.86
3   East  894.95
4   East  391.91

Single value: 510.88


#### Example 4: Selecting with Conditions (Preview of Filtering)

In [15]:
# Rows where Sales > 500
high_sales = df.loc[df['Sales'] > 500, ['Product', 'Sales']]
print(high_sales.head())

     Product   Sales
0   Widget C  510.88
3   Widget B  894.95
7   Widget C  916.15
9   Widget B  682.92
14  Widget A  580.68


### 3. Filtering the Data
Explanation: Filtering narrows down rows based on conditions (e.g., values > threshold). Use boolean masks, queries, or methods like `isin`. Always returns a view/copy—use `.copy()` if modifying.
Key Tips:

Combine conditions with `&` (and), `|` (or), `~` (not)—wrap in parentheses!
`query()` is string-based and readable for complex filters.
For multiple columns: chain conditions.

Example 1: Boolean Indexing (Basic Mask)
Create a True/False Series and pass to `loc` or `[]`.

In [17]:
# Filter rows where Sales > 500
mask = df['Sales'] > 500
filtered = df.loc[mask]
print("Filtered shape:", filtered.shape)
filtered[['Region', 'Sales']].head()

Filtered shape: (55, 7)


,Region,Sales
0,East,510.88
3,East,894.95
7,North,916.15
9,South,682.92
14,West,580.68


In [18]:
# Multiple conditions: Sales > 500 AND Region == 'North'
mask_multi = (df['Sales'] > 500) & (df['Region'] == 'North')
filtered_multi = df.loc[mask_multi]
print("\nMulti-condition shape:", filtered_multi.shape)


Multi-condition shape: (15, 7)


Alternative Ways:

- Direct: df[df['Sales'] > 500]
- Negate: df.loc[~(df['Sales'] > 500)] (Sales <= 500)

#### Example 2: Using query() Method
String-based, great for readability.

In [19]:
## Simple query
high_sales_query = df.query('Sales > 500')
print(high_sales_query[['Product', 'Sales']].head())

     Product   Sales
0   Widget C  510.88
3   Widget B  894.95
7   Widget C  916.15
9   Widget B  682.92
14  Widget A  580.68


In [20]:
# With variables: threshold = 500
threshold = 500
high_sales_var = df.query('Sales > @threshold and Region == "North"')
print("\nWith variable shape:", high_sales_var.shape)


With variable shape: (15, 7)


Alternative Ways:

- String interpolation: `df.query(f'Sales > {threshold}')`
- For strings: `df.query('Region in ["North", "South"]')`

#### Example 3: isin() for Categorical Filtering
Check if values are in a list.

In [21]:
# Filter Products in specific list
products_list = ['Widget A', 'Widget B']
filtered_isin = df[df['Product'].isin(products_list)]
print("ISIN shape:", filtered_isin.shape)
filtered_isin.head(2)[['Product', 'Quantity']]

ISIN shape: (61, 7)


,Product,Quantity
1,Widget B,16
2,Widget B,8


In [22]:
# Combined with boolean
mask_isin = df['Region'].isin(['East', 'West']) & (df['Discount'] > 0)
filtered_combined = df.loc[mask_isin]
print("\nCombined shape:", filtered_combined.shape)


Combined shape: (38, 7)


#### Example 4: between() for Ranges
Filter numeric/date ranges.

In [23]:
# Sales between 200 and 600
between_sales = df[df['Sales'].between(200, 600)]
print("Between shape:", between_sales.shape)

Between shape: (52, 7)


In [24]:
# Date range
date_mask = df['Date'].between('2023-01-10', '2023-01-20')
filtered_dates = df[date_mask]
print("\nDate filter shape:", filtered_dates.shape)


Date filter shape: (11, 7)


__Practice Exercise__: Filter for 'South' region with Quantity > 10, using both boolean indexing and query(). Count the rows with len().

### 4. Manipulating the Data
Explanation: Manipulation involves transforming data: creating new columns, applying functions, replacing values, or handling missing data. Use vectorized operations for speed—avoid loops!
Key Tips:

- Vectorized: `df['New'] = df['A'] * 2 `(fast).
- `apply()` is flexible but slower; use map() for Series.
- For NaNs: `fillna()`, `dropna()`.

#### Example 1: Creating New Columns
Add derived features.

In [26]:
# Simple arithmetic
df['Total_Cost'] = df['Sales'] * df['Quantity']
print("New column added. Head:\n", df[['Sales', 'Quantity', 'Total_Cost']].head())

New column added. Head:
     Sales  Quantity  Total_Cost
0  510.88        14     7152.32
1  296.60        16     4745.60
2  474.86         8     3798.88
3  894.95         7     6264.65
4  391.91         3     1175.73


In [27]:
# Conditional: High/Low Sales
df['Sales_Category'] = np.where(df['Sales'] > 500, 'High', 'Low')
print("\nConditional column:\n", df['Sales_Category'].value_counts())


Conditional column:
 Sales_Category
High    55
Low     45
Name: count, dtype: int64


Alternative Ways:

- `df.assign(New_Col = df['A'] + df['B'])` (non-destructive, returns new DF)
- Lambda: `df`['Squared'] = df['Sales'].apply(lambda x: x**2)`

#### Example 2: Applying Functions
Transform columns with custom logic.

In [28]:
# Apply to single column (map for Series)
df['Sales_Rounded'] = df['Sales'].map(lambda x: round(x, 0))
print("Rounded Sales:\n", df[['Sales', 'Sales_Rounded']].head())

Rounded Sales:
     Sales  Sales_Rounded
0  510.88          511.0
1  296.60          297.0
2  474.86          475.0
3  894.95          895.0
4  391.91          392.0


In [29]:
# Apply to entire row (axis=1)
df['Region_Product'] = df.apply(lambda row: f"{row['Region']}_{row['Product']}", axis=1)
print("\nCombined column:\n", df['Region_Product'].head())


Combined column:
 0     East_Widget C
1     West_Widget B
2    North_Widget B
3     East_Widget B
4     East_Widget B
Name: Region_Product, dtype: object


#### Example 3: Replacing Values
Fix errors or standardize.

In [30]:
# Replace specific values
df_replace = df.copy()
df_replace['Product'] = df_replace['Product'].replace('Widget A', 'Widget X')
print("Replaced:\n", df_replace['Product'].unique())

Replaced:
 ['Widget C' 'Widget B' 'Widget X']


__Practice Exercise__: Create a 'Profit' column as Revenue - (Sales * 0.2 * Quantity). Then apply a function to categorize Profit as 'Profitable' if > 1000 else 'Loss'.

### 5. Sorting the Data
Explanation: Sorting arranges data by one or more columns (ascending/descending). Use `sort_values()` for columns or `sort_index()` for indices. Resets order for analysis.
Key Tips:

- `inplace=True` modifies original; default returns copy.
- Multi-level sort: list of columns.
- Stable sort preserves relative order.

#### Example 1: Sorting by Single Column
Basic ascending/descending.

In [32]:
# Sort by Sales ascending
sorted_sales_asc = df.sort_values('Sales')
print("Ascending head:\n", sorted_sales_asc['Sales'].head())

Ascending head:
 10    100.47
99    116.27
33    129.75
23    158.40
41    172.77
Name: Sales, dtype: float64


In [33]:
# Descending
sorted_sales_desc = df.sort_values('Sales', ascending=False)
print("\nDescending head:\n", sorted_sales_desc['Sales'].head())


Descending head:
 29    997.97
74    996.63
97    995.01
46    985.08
71    983.95
Name: Sales, dtype: float64


#### Example 2: Sorting by Multiple Columns
Sort by Region then Sales.

In [34]:
# Multi-column: Region asc, Sales desc
multi_sorted = df.sort_values(['Region', 'Sales'], ascending=[True, False])
print("Multi-sorted head:\n", multi_sorted[['Region', 'Sales']].head(10))

Multi-sorted head:
    Region   Sales
60   East  946.47
84   East  898.99
3    East  894.95
85   East  865.84
80   East  730.32
52   East  726.23
64   East  708.12
40   East  633.43
39   East  597.60
0    East  510.88


### 6. Grouping the Data
Explanation: Grouping aggregates data by categories (e.g., mean sales per region). Use `groupby()` followed by aggregation (`agg()`, `mean()`, etc.). Handles splits, applies, combines.
Key Tips:

- Common aggs: `sum()`, `mean()`, `count()`, `max()`.
- Multiple aggs: `agg({'col1': 'mean', 'col2': 'sum'})`.
- For custom: `apply()` or `transform()` (keeps shape).

#### Example 1: Basic GroupBy with Aggregation
Group by one column.

##### Add a column for each wine's alcohol deviation from its class mean.

In [35]:
# Mean Revenue per Region
grouped_region = df.groupby('Region')['Revenue'].mean()
print("Mean Revenue per Region:\n", grouped_region)

Mean Revenue per Region:
 Region
East     3933.709979
North    4832.290875
South    6124.784673
West     5128.797900
Name: Revenue, dtype: float64


In [36]:
# Count per Product
count_prod = df.groupby('Product').size()  # or .count()
print("\nCounts per Product:\n", count_prod)


Counts per Product:
 Product
Widget A    35
Widget B    26
Widget C    39
dtype: int64


Alternative Ways:

- `df['Region'].value_counts()` (quick counts)
- Multiple columns: `df.groupby(['Region', 'Product'])['Revenue'].mean()`

#### Example 2: Multiple Aggregations
Apply different functions.

In [37]:
# Agg on Revenue: mean and sum per Region
agg_multi = df.groupby('Region')['Revenue'].agg(['mean', 'sum', 'count'])
print("Multi-agg:\n", agg_multi)

Multi-agg:
                mean          sum  count
Region                                 
East    3933.709979   94409.0395     24
North   4832.290875   96645.8175     20
South   6124.784673  159244.4015     26
West    5128.797900  153863.9370     30


#### Example 3: GroupBy with Transform/Apply
Transform keeps original shape; apply for custom.

In [38]:
# Transform: Rank within group
df['Rev_Rank_in_Region'] = df.groupby('Region')['Revenue'].transform('rank', ascending=False)
print("With ranks head:\n", df[['Region', 'Revenue', 'Rev_Rank_in_Region']].head())

With ranks head:
   Region   Revenue  Rev_Rank_in_Region
0   East  6794.704                 2.0
1   West  4745.600                14.0
2  North  3608.936                10.0
3   East  5638.185                 7.0
4   East  1058.157                21.0


In [39]:
# Apply custom function
def top_n(group, n=3):
    return group.nlargest(n, 'Revenue')

top_per_region = df.groupby('Region').apply(top_n)
print("\nTop 3 per Region shape:", top_per_region.shape)


Top 3 per Region shape: (12, 13)


C:\Users\hp\AppData\Local\Temp\ipykernel_7988\3579434290.py:5: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top_per_region = df.groupby('Region').apply(top_n)


#### Example 4: Grouping by Date (Resample for Time Series)
Group by month.

In [40]:
# Set Date as index if needed
df.set_index('Date', inplace=True)

# Resample monthly sum
monthly_sales = df.groupby(df.index.to_period('M'))['Revenue'].sum()
print("Monthly Revenue:\n", monthly_sales.head())

Monthly Revenue:
 Date
2023-01    140809.0580
2023-02    128298.5860
2023-03    170612.2170
2023-04     64443.3345
Freq: M, Name: Revenue, dtype: float64


__Practice Exercise__: Group by 'Product' and compute median Quantity and max Discount. Use agg() with a dictionary.

### 7. Ranking the Data
Explanation: Ranking assigns order (1, 2, 3...) to values within groups or overall. Useful for percentiles or competitions. `rank()` method handles ties.
Key Tips:

`method='average'`, `'min'`, `'dense'`, etc., for ties.
Grouped: Use with `groupby()`.
Percentile: `pct=True`.

#### Example 1: Basic Ranking
Rank entire column.

In [41]:
# Rank Sales ascending (lowest=1)
df['Sales_Rank'] = df['Sales'].rank(method='average')
print("Ranks head:\n", df[['Sales', 'Sales_Rank']].head())

# Descending (highest=1)
df['Sales_Rank_Desc'] = df['Sales'].rank(ascending=False, method='min')
print("\nDescending head:\n", df[['Sales', 'Sales_Rank_Desc']].head())

Ranks head:
              Sales  Sales_Rank
Date                          
2023-01-01  510.88        48.0
2023-01-02  296.60        16.0
2023-01-03  474.86        45.0
2023-01-04  894.95        87.0
2023-01-05  391.91        32.0

Descending head:
              Sales  Sales_Rank_Desc
Date                               
2023-01-01  510.88             53.0
2023-01-02  296.60             85.0
2023-01-03  474.86             56.0
2023-01-04  894.95             14.0
2023-01-05  391.91             69.0


Alternative Ways:

- Dense: `rank(method='dense')` (no gaps in ranks)
- Percentile: `df['Sales'].rank(pct=True)` (0-1 scale)

#### Example 2: Grouped Ranking
Rank within groups.

In [42]:
# Rank Revenue within each Region
df['Rev_Rank_Region'] = df.groupby('Region')['Revenue'].rank(ascending=False, method='dense')
print("Grouped ranks head:\n", df[['Region', 'Revenue', 'Rev_Rank_Region']].head(10))

Grouped ranks head:
            Region     Revenue  Rev_Rank_Region
Date                                          
2023-01-01   East   6794.7040              2.0
2023-01-02   West   4745.6000             14.0
2023-01-03  North   3608.9360             10.0
2023-01-04   East   5638.1850              7.0
2023-01-05   East   1058.1570             21.0
2023-01-06   West   3567.9600             18.0
2023-01-07  North    357.5695             20.0
2023-01-08  North  14658.4000              2.0
2023-01-09   East   4139.0400             12.0
2023-01-10  South  12326.7060              2.0


Alternative: Within Product: df.groupby('Product')['Quantity'].rank()

#### Example 3: Ranking with Ties and Custom
Handle specifics.

In [43]:
# Min method for ties (all get lowest rank)
df['Quantity_Rank_Min'] = df['Quantity'].rank(method='min')
print("Min ties:\n", df[df['Quantity'] == 5][['Quantity', 'Quantity_Rank_Min']])  # Example ties

# First occurrence
df['Sales_Rank_First'] = df['Sales'].rank(method='first')

Min ties:
             Quantity  Quantity_Rank_Min
Date                                   
2023-01-21         5               25.0
2023-02-02         5               25.0
2023-03-21         5               25.0
2023-03-26         5               25.0
2023-04-10         5               25.0


Alternative: method='max' (highest rank for ties)

#### Example 4: Overall DataFrame Ranking
Rank across multiple columns.

In [45]:
# Rank by multiple: Create composite score
df['Composite'] = df['Sales'] + df['Revenue']
df['Composite_Rank'] = df['Composite'].rank(ascending=False)
top_ranked = df.nlargest(5, 'Composite_Rank')[['Product', 'Composite']]
print("Top 5 ranked:\n", top_ranked)

Top 5 ranked:
              Product  Composite
Date                           
2023-01-20  Widget A   465.2010
2023-01-18  Widget A   633.5695
2023-02-11  Widget B   665.1645
2023-01-24  Widget C   696.9600
2023-04-10  Widget B   697.6200


__Practice Exercise__: Add a rank column for Quantity within each Product group, using 'dense' method. Display the top 3 per product using `groupby().apply(lambda x: x.nlargest(3, 'Quantity_Rank'))`.